
## 1. Import The ImageDataGenerator Library

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 2. Configure ImageDataGenerator Class
## Image Data Augmentation

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.1,
zoom_range = 0.1,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

## 3. Apply ImageDataGenerator Functionality To Trainset And Testset

In [4]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/body/training',target_size = (224, 224),batch_size = 10,class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/body/validation',target_size = (224, 224),batch_size = 10,class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


## MODEL BUILDING
### 1. Importing The Model Building Libraries

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

### 2. Loading The Model

In [6]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/body/training'
valid_path = '/content/drive/MyDrive/body/validation'
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


### 3. Adding Flatten Layer

In [7]:
for layer in vgg16.layers:layer.trainable = False

In [8]:
folders = glob('/content/drive/MyDrive/body/training/*')
folders

['/content/drive/MyDrive/body/training/02-side',
 '/content/drive/MyDrive/body/training/00-front',
 '/content/drive/MyDrive/body/training/01-rear']

In [9]:
x = Flatten()(vgg16.output)
len(folders)

3

### 4. Adding Output Layer

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

### 5. Creating A Model Object

In [11]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### 6. Configure The Learning Process

In [13]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

### 7. Train The Model

In [21]:
r = model.fit_generator(
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


98/98 [==============================] - 292s 3s/step - loss: 0.9919 - accuracy: 0.5863 - val_loss: 0.9702 - val_accuracy: 0.6550
Epoch 2/5
98/98 [==============================] - 14s 144ms/step - loss: 0.6779 - accuracy: 0.7354 - val_loss: 1.0075 - val_accuracy: 0.6374
Epoch 3/5
98/98 [==============================] - 14s 145ms/step - loss: 0.5014 - accuracy: 0.8029 - val_loss: 1.8027 - val_accuracy: 0.4971
Epoch 4/5
98/98 [==============================] - 15s 148ms/step - loss: 0.3898 - accuracy: 0.8580 - val_loss: 0.9130 - val_accuracy: 0.6959
Epoch 5/5
98/98 [==============================] - 15s 154ms/step - loss: 0.2440 - accuracy: 0.9224 - val_loss: 1.0812 - val_accuracy: 0.6901


### 8. Save The Model

In [15]:
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/ibm project/Intelligent Vehicle Damage Assessment & Cost Estimator/MODEL/BODY.h5')

### 9. Test The Model

In [16]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [17]:
model = load_model('/content/drive/MyDrive/ibm project/Intelligent Vehicle Damage Assessment & Cost Estimator/MODEL/BODY.h5')

In [18]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  if(np.max(img)>1):
    img = img/255.0
    img = np.array([img])
    prediction = model.predict(img)
    label = ["front","rear","side"]
    preds = label[np.argmax(prediction)]
    return preds

In [19]:
data = "/content/drive/MyDrive/body/training/00-front/0007.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 812ms/step
front
